# Données

In [13]:
import os
import requests
from scipy.io import loadmat
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import OneHotEncoder
import numpy as np

def get_mnist(): 
    mnist = fetch_openml('mnist_784')
    X = mnist['data'].values
    X = (mnist['data'] > 128).astype(int)
    y = mnist['target'].values
    y = OneHotEncoder().fit_transform(y.reshape(-1, 1)).toarray()

    return X, y
    
def get_alphadigs():
    if os.path.exists('data/alphadigs.mat'):
        return loadmat('data/alphadigs.mat')
    
    alphadigs_url = 'https://cs.nyu.edu/~roweis/data/binaryalphadigs.mat'
    r = requests.get(alphadigs_url, allow_redirects=True)

    with open('data/alphadigs.mat', 'wb') as f:
        f.write(r.content)
                
    return loadmat('data/alphadigs.mat')
      
def lire_alpha_digit(list_digits) : 
    X = get_alphadigs()
    indices = np.where(np.isin(X["classlabels"], list_digits))[1]
    rows = X["dat"][indices]
    return rows  

def lire_alpha_digit1(dataset, list_idx):
  imgs_set = dataset[list_idx,:].flatten()
  imgs_flatten = []

  for img in imgs_set:
    imgs_flatten.append(img.flatten())
  
  imgs_set_output = np.array(imgs_flatten)

  return imgs_set_output

In [3]:
f = lire_alpha_digit(['0', '2', 'P','F', 'Z'])

/Users/zakariatozy/opt/anaconda3/lib/python3.9/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [4]:
import numpy as np
import numpy as np
import random
import matplotlib.pyplot as plt

def relu(x):
    return np.maximum(x, 0)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Fonctions élémentaires

## Construction d’un RBM et test sur Binary AlphaDigits

In [38]:
class RBM:
    def __init__(self, p, q):
        self.W = 0.01 * np.random.rand(p,q)
        self.a = np.zeros((1,p))
        self.b = np.zeros((1,q))
        
def init_RBM(p, q):
    return RBM(p, q)

In [47]:
def entree_sortie_RBM(rbm, x):
    return sigmoid(np.dot(x, rbm.W) + rbm.b)

In [39]:
def sortie_entree_RBM(rbm, y): #(n * q) -> (n * p)
    return sigmoid(np.dot(y, rbm.W.T) + rbm.a)

In [56]:
def train_RBM(rbm,X,epsilon, epochs, batch_size):
        
    n = X.shape[0]
    p = X.shape[1]
    q = len(rbm.b) 
    
    for epoch in range(epochs):
        X_copy = X.copy()
        
        for batch  in range(0, n, batch_size):
            X_batch = X_copy[batch:batch + batch_size]
            
            tb = len(X_batch)
            
            v_0 = X_batch # (tb * p)
            print(tb,p,v_0.shape)
            
            p_h_v_0 = entree_sortie_RBM(rbm,v_0) # (tb * q)
            print(tb,q,p_h_v_0.shape)
            
            h_0 = (p_h_v_0 > np.random.rand(tb, q)).astype(int) # (tb * q)
            print(tb,q,h_0.shape)
            
            p_v_h_0 = sortie_entree_RBM(rbm,h_0) # (tb * p)
            print(tb,p,p_v_h_0.shape)
            v_1 = (p_v_h_0 > np.random.rand(tb, p)).astype(int) # (tb * p)
            print(tb,p,v_1.shape)
            
            p_h_v_1 = entree_sortie_RBM(rbm,v_1) # (tb * q)
            print(tb,q,p_h_v_1.shape)
            
            norm_epsilon = epsilon / tb
            
            rbm.W += norm_epsilon * np.dot(v_0.T, p_h_v_0) - np.dot(v_1.T, p_h_v_1) # (p * q)   
            print(p,q,rbm.W.shape)     
            rbm.a += norm_epsilon * np.sum(v_0 - v_1, axis=0) # (1 * p)
            print(p,rbm.a.shape)
            rbm.b += norm_epsilon * np.sum(p_h_v_0 - p_h_v_1, axis=0) # (1 * q)
            print(q,rbm.b.shape)
            
    return RBM


In [ ]:
import scipy.io as sio
mat_content = sio.loadmat("data/alphadigs.mat")

In [57]:
X = lire_alpha_digit1(dataset=mat_content["dat"], list_idx=list_idx)
print(type(X))
p = len(X[0])
q = 100
Rbm = init_RBM(p,q)
Rbm = train_RBM(Rbm,X,0.1, 10, 10)

<class 'numpy.ndarray'>
10 320 (10, 320)
10 1 (10, 100)
10 1 (10, 100)
10 320 (10, 320)
10 320 (10, 320)
10 1 (10, 100)
320 1 (320, 100)
320 (1, 320)
1 (1, 100)
10 320 (10, 320)
10 1 (10, 100)
10 1 (10, 100)
10 320 (10, 320)
10 320 (10, 320)
10 1 (10, 100)
320 1 (320, 100)
320 (1, 320)
1 (1, 100)
10 320 (10, 320)
10 1 (10, 100)
10 1 (10, 100)
10 320 (10, 320)
10 320 (10, 320)
10 1 (10, 100)
320 1 (320, 100)
320 (1, 320)
1 (1, 100)
10 320 (10, 320)
10 1 (10, 100)
10 1 (10, 100)
10 320 (10, 320)
10 320 (10, 320)
10 1 (10, 100)
320 1 (320, 100)
320 (1, 320)
1 (1, 100)
10 320 (10, 320)
10 1 (10, 100)
10 1 (10, 100)
10 320 (10, 320)
10 320 (10, 320)
10 1 (10, 100)
320 1 (320, 100)
320 (1, 320)
1 (1, 100)
10 320 (10, 320)
10 1 (10, 100)
10 1 (10, 100)
10 320 (10, 320)
10 320 (10, 320)
10 1 (10, 100)
320 1 (320, 100)
320 (1, 320)
1 (1, 100)
10 320 (10, 320)
10 1 (10, 100)
10 1 (10, 100)
10 320 (10, 320)
10 320 (10, 320)
10 1 (10, 100)
320 1 (320, 100)
320 (1, 320)
1 (1, 100)
10 320 (10, 320)


/var/folders/2m/nlw4h6h974767p9gyt3k742h0000gn/T/ipykernel_77165/1591538201.py:9: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


10 320 (10, 320)
10 1 (10, 100)
320 1 (320, 100)
320 (1, 320)
1 (1, 100)
10 320 (10, 320)
10 1 (10, 100)
10 1 (10, 100)
10 320 (10, 320)
10 320 (10, 320)
10 1 (10, 100)
320 1 (320, 100)
320 (1, 320)
1 (1, 100)
10 320 (10, 320)
10 1 (10, 100)
10 1 (10, 100)
10 320 (10, 320)
10 320 (10, 320)
10 1 (10, 100)
320 1 (320, 100)
320 (1, 320)
1 (1, 100)
10 320 (10, 320)
10 1 (10, 100)
10 1 (10, 100)
10 320 (10, 320)
10 320 (10, 320)
10 1 (10, 100)
320 1 (320, 100)
320 (1, 320)
1 (1, 100)
10 320 (10, 320)
10 1 (10, 100)
10 1 (10, 100)
10 320 (10, 320)
10 320 (10, 320)
10 1 (10, 100)
320 1 (320, 100)
320 (1, 320)
1 (1, 100)
10 320 (10, 320)
10 1 (10, 100)
10 1 (10, 100)
10 320 (10, 320)
10 320 (10, 320)
10 1 (10, 100)
320 1 (320, 100)
320 (1, 320)
1 (1, 100)
7 320 (7, 320)
7 1 (7, 100)
7 1 (7, 100)
7 320 (7, 320)
7 320 (7, 320)
7 1 (7, 100)
320 1 (320, 100)
320 (1, 320)
1 (1, 100)


In [ ]:
# Generate 25 new images using the RBM
generated_images = generer_image_RBM(rbm, 100, 25)

# Display the generated images
afficher_images(generated_images)

def generer_images_RBM(rbm, iterations, num_images):
    hidden_probs = np.random.rand(num_images, rbm.W.shape[1])
    hidden_states = hidden_probs > 0.5

    for i in range(iterations):
        visible_probs = sortie_entree_RBM(rbm, hidden_states)
        visible_states = visible_probs > np.random.rand(*visible_probs.shape)
        hidden_probs = entree_sortie_RBM (rbm, visible_states)
        hidden_states = hidden_probs > np.random.rand(*hidden_probs.shape)

    return visible_states

def afficher_images(images, n_cols=None):
    n_images, n_pixels = images.shape
    if n_cols is None:
        n_cols = int(np.ceil(np.sqrt(n_images)))